In [1]:
from loguru import logger
import snoop
import birdseye
import numpy as np
%load_ext blackcellmagic

In [2]:
from statsmodels.stats.weightstats import ztest
from scipy.stats import f, ttest_ind
from birdseye import eye

Постройте в Питоне функцию, которая будет проводить анализ выборки. На вход она берёт:

- один или два ряда
- число m для проведения гипотезы о равенстве среднего числу (для каждого ряда)
- число diff_means, которое по умолчанию равно нулю, которое показывает гипотетическую разницу средних в двухвыборочном тесте
- уровень значимости для проведения всех стат.тестов

и делает следующие выводы:

1. Для каждого ряда она выводит (с помощью print и расшифровки) среднее, дисперсию, стандартное отклонение.
2. Для каждого ряда она выводит квантильный анализ: медиану, квартили, межквартильный размах и децили.
3. Если выборки две, то результат проведения одновыборочного z-теста (на равенство средних в обеих выборках числу m), двухвыборочного z-теста на равенство средних, t-теста на равенство средних и F-теста на равенство дисперсий для двух выборок. Результаты тестов надо описать в print понятным языком

(*) Постарайтесь разбросать подзадачи на дополнительные функции

- Одна функция для отображения результатов теста, как z, так и t и F (с проверкой p_value и принтом формулировки результата)
- Одна функция для выполнения F-теста (нахождение значения F-критерия и проверки его по распределению Фишера)
- Хорошо бы, чтобы результаты по отображению результатов F-теста были пайплайном из трёх функций

In [3]:
arr1 = np.random.uniform(1, 100, 20).astype(int)
arr2 = np.random.poisson(10, 20)
np.mean(arr1)

37.0

In [4]:
def calculate_fisher(*arrs, alpha):
    sigma_1_squared = np.std(arrs[0]) ** 2
    n_1 = len(arrs[0])
    sigma_2_squared = np.std(arrs[1]) ** 2
    n_2 = len(arrs[1])
    if sigma_1_squared > sigma_2_squared:
        fisher_rv = f(n_1 - 1, n_2 - 1)
        f_score = sigma_1_squared / sigma_2_squared
    else:
        fisher_rv = f(n_2 - 1, n_1 - 1)
        f_score = sigma_2_squared / sigma_1_squared
    test_eq = f_score < fisher_rv.ppf(alpha / 2) or f_score > fisher_rv.ppf(
        1 - alpha / 2
    )
    if test_eq == True:
        # альтернативная гипотеза – sigma_1_squared > sigma_2_squared
        test_neq = f_score > fisher_rv.ppf(1 - alpha)
        return test_eq, f_score, test_neq
    else:
        return test_eq, f_score,
    

In [5]:
@eye
def calculate_stats(*arrs, m, alpha, diff_means=0):
    i = len(arrs)
    for series in arrs:
        series = np.array(series)
        print(f"Printing stats for {series}")
        print(
            np.mean(series),
            "Mean\n",
            np.var(series),
            "Variance\n",
            np.std(series),
            "Standard Dev",
            #sep=' '
        )
        print(
            np.quantile(series, 0.5),
            " Median\n",
            np.quantile(series, 0.25),
            "Quantile 0.25\n",
            np.quantile(series, 0.75),
            "Quantile 0.75",
        )
        print(np.quantile(series, 0.75) - np.quantile(series, 0.25), "IQ Distance")
        print("_" * 20)
        if i == 2:
            print("Single-sample Z-test")
            t, p_value = ztest(series, value=m)
            if p_value >= 0.05:
                print("H0 accepted")
            else:
                print("H0 rejected")
            print("_" * 20)
    if i == 2:
        print("Two-sample Z-test")
        tstat, p_value = ztest(arrs[0], arrs[1])
        if p_value >= 0.05:
            print("H0 accepted (means are equiv)")
        else:
            print("H0 rejected (means are not equiv)")
        print("_" * 20)
        print("T-test")
        tstat, p_value = ttest_ind(arrs[0], arrs[1])
        if p_value >= 0.05:
            print("H0 accepted (means are equiv)")
        else:
            print("H0 rejected (means are not equiv)")
        print("_" * 20)
        print("Fisher-test")
        res, *_ = calculate_fisher(*arrs, alpha=alpha)
        if not res:
            print("Vars equal")
        else:
            print("Vars are not equal")

In [6]:
calculate_stats(arr1, arr2, m=51, alpha=0.05)

Printing stats for [34 14 14 15 59 36 24 75 38 55 27 88 22  4 43 17 28 48 68 31]
37.0 Mean
 481.2 Variance
 21.93627133311402 Standard Dev
32.5  Median
 20.75 Quantile 0.25
 49.75 Quantile 0.75
29.0 IQ Distance
____________________
Single-sample Z-test
H0 rejected
____________________
Printing stats for [ 5  8 10 12 12 12 10 14 10 14 12  5 12  4 16 13  8 11 18  7]
10.65 Mean
 12.8275 Variance
 3.5815499438092444 Standard Dev
11.5  Median
 8.0 Quantile 0.25
 12.25 Quantile 0.75
4.25 IQ Distance
____________________
Single-sample Z-test
H0 rejected
____________________
Two-sample Z-test
H0 rejected (means are not equiv)
____________________
T-test
H0 rejected (means are not equiv)
____________________
Fisher-test
Vars are not equal
